# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
sys.path.append("../../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
filename = "csv/mypy_testset.csv"
df = pd.read_csv(filename)
df

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.142706e+10,IssuesEvent,2020-02-03 23:33:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Type of conditional expression is object,false-positive needs discussion priority-1-nor...,The type of the following conditional expressi...
1,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,bug false-positive priority-0-high,While interacting with `zip` I encountered a v...
2,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,feature,"After #5870 is done, it would be nice to also ..."
3,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,bug,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...
4,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,feature,**Feature**\r\n\r\nWhen the implementation add...
...,...,...,...,...,...,...,...,...,...
5009,2.060497e+10,IssuesEvent,2022-03-06 20:54:33,python/mypy,https://api.github.com/repos/python/mypy,closed,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
5010,2.060505e+10,IssuesEvent,2022-03-06 21:09:50,python/mypy,https://api.github.com/repos/python/mypy,reopened,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
5011,2.060518e+10,IssuesEvent,2022-03-06 21:33:34,python/mypy,https://api.github.com/repos/python/mypy,opened,"Getting Invalid ""type: ignore"" comment reports...",bug,**Bug Report**\r\n\r\n3rd party library I am u...
5012,2.060562e+10,IssuesEvent,2022-03-06 23:01:04,python/mypy,https://api.github.com/repos/python/mypy,closed,Wrong type inference with unpacking and enumer...,bug priority-1-normal,"Mypy infers that the type of `[*enumerate([""x""..."


In [3]:
# Number or different labels
df.labels.value_counts().to_frame()[:50]

,count
labels,
bug,2266
feature,443
crash,190
question,93
documentation,89
bug priority-1-normal,52
bug priority-0-high,46
crash priority-0-high,39
needs discussion,39


In [4]:
# Split data based on labels. Contains P0 and P1 in one dataframe, the rest in another

pattern = 'priority-0-high'
# Check if 'labels' contains the pattern
hp = df[df['labels'].str.contains(pattern)]
# Reset index
hp = hp.reset_index(drop=True)
hp


,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,bug false-positive priority-0-high,While interacting with `zip` I encountered a v...
1,2.655427e+10,IssuesEvent,2023-01-20 10:36:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Unbound type variable false positive with six....,bug topic-type-variables priority-0-high,This code generates a false positive:\r\n```py...
2,2.656236e+10,IssuesEvent,2023-01-20 16:54:26,python/mypy,https://api.github.com/repos/python/mypy,closed,Unbound type variable false positive with six....,bug topic-type-variables priority-0-high,This code generates a false positive:\r\n```py...
3,5.439713e+09,IssuesEvent,2017-03-06 14:10:37,python/mypy,https://api.github.com/repos/python/mypy,closed,Crash in super() outside a method,crash priority-0-high,The simplest repro:\r\n```python\r\nclass C:\r...
4,8.708969e+09,IssuesEvent,2018-12-06 12:35:33,python/mypy,https://api.github.com/repos/python/mypy,closed,TypedDict missing many dict methods,bug false-positive priority-0-high topic-typed...,mypy isn't recognizing methods of TypedDict.\r...
...,...,...,...,...,...,...,...,...,...
434,1.003922e+10,IssuesEvent,2019-07-18 16:50:07,python/mypy,https://api.github.com/repos/python/mypy,opened,Support error codes and ignoring only specific...,feature priority-0-high,It would be handy to support error codes to al...
435,2.322156e+10,IssuesEvent,2022-08-02 18:45:12,python/mypy,https://api.github.com/repos/python/mypy,closed,Should concrete implementations be required to...,bug priority-0-high topic-protocols,"* Are you reporting a bug, or opening a featur..."
436,8.837451e+09,IssuesEvent,2019-01-05 04:59:35,python/mypy,https://api.github.com/repos/python/mypy,closed,Windows dmypy CI flakes,crash priority-0-high topic-daemon,We've seen some failures of the daemon tests o...
437,8.527242e+09,IssuesEvent,2018-11-02 18:50:03,python/mypy,https://api.github.com/repos/python/mypy,closed,Add plugin API to specify fine grained depende...,priority-0-high topic-fine-grained-incremental...,Currently plugins can add type information tha...


In [5]:
# Remove pattern from df
random = df[~df['labels'].str.contains(pattern)]
random = random.reset_index(drop=True)
random

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.142706e+10,IssuesEvent,2020-02-03 23:33:27,python/mypy,https://api.github.com/repos/python/mypy,closed,Type of conditional expression is object,false-positive needs discussion priority-1-nor...,The type of the following conditional expressi...
1,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,feature,"After #5870 is done, it would be nice to also ..."
2,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,bug,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...
3,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,feature,**Feature**\r\n\r\nWhen the implementation add...
4,7.238785e+09,IssuesEvent,2018-02-13 15:37:22,python/mypy,https://api.github.com/repos/python/mypy,closed,Typechecking attrs-generated classes,feature needs discussion topic-plugins,[attrs](https://github.com/hynek/attrs) remove...
...,...,...,...,...,...,...,...,...,...
4570,2.060497e+10,IssuesEvent,2022-03-06 20:54:33,python/mypy,https://api.github.com/repos/python/mypy,closed,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
4571,2.060505e+10,IssuesEvent,2022-03-06 21:09:50,python/mypy,https://api.github.com/repos/python/mypy,reopened,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
4572,2.060518e+10,IssuesEvent,2022-03-06 21:33:34,python/mypy,https://api.github.com/repos/python/mypy,opened,"Getting Invalid ""type: ignore"" comment reports...",bug,**Bug Report**\r\n\r\n3rd party library I am u...
4573,2.060562e+10,IssuesEvent,2022-03-06 23:01:04,python/mypy,https://api.github.com/repos/python/mypy,closed,Wrong type inference with unpacking and enumer...,bug priority-1-normal,"Mypy infers that the type of `[*enumerate([""x""..."


In [6]:
# Removing P1 and P2 since these are medium and low priority
pattern = 'priority-1-normal|priority-2-low'
# Remove pattern from df
random = random[~random['labels'].str.contains(pattern)]
random = random.reset_index(drop=True)
random

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,feature,"After #5870 is done, it would be nice to also ..."
1,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,bug,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...
2,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,feature,**Feature**\r\n\r\nWhen the implementation add...
3,7.238785e+09,IssuesEvent,2018-02-13 15:37:22,python/mypy,https://api.github.com/repos/python/mypy,closed,Typechecking attrs-generated classes,feature needs discussion topic-plugins,[attrs](https://github.com/hynek/attrs) remove...
4,3.912026e+09,IssuesEvent,2016-04-20 08:51:12,python/mypy,https://api.github.com/repos/python/mypy,closed,Improve unification of conditional expressions,bug,I have this snippet of code\r\n\r\n```python\r...
...,...,...,...,...,...,...,...,...,...
3981,2.060491e+10,IssuesEvent,2022-03-06 20:42:52,python/mypy,https://api.github.com/repos/python/mypy,opened,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
3982,2.060497e+10,IssuesEvent,2022-03-06 20:54:33,python/mypy,https://api.github.com/repos/python/mypy,closed,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
3983,2.060505e+10,IssuesEvent,2022-03-06 21:09:50,python/mypy,https://api.github.com/repos/python/mypy,reopened,Color output missing when errors redrected (wi...,bug,<!--\r\n If you're new to mypy and you're not...
3984,2.060518e+10,IssuesEvent,2022-03-06 21:33:34,python/mypy,https://api.github.com/repos/python/mypy,opened,"Getting Invalid ""type: ignore"" comment reports...",bug,**Bug Report**\r\n\r\n3rd party library I am u...


In [7]:
random.labels.value_counts().to_frame()[:50]

,count
labels,
bug,2266
feature,443
crash,190
question,93
documentation,89
needs discussion,39
bug topic-paramspec,26
bug topic-type-narrowing,18
bug topic-stubtest,16


In [8]:
# Number or different labels
hp.labels.value_counts().to_frame()[:50]

,count
labels,
bug priority-0-high,46
crash priority-0-high,39
feature priority-0-high,22
bug false-positive priority-0-high,19
priority-0-high,16
documentation priority-0-high,11
crash new-semantic-analyzer priority-0-high,9
crash priority-0-high topic-fine-grained-incremental,9
priority-0-high topic-usability,7


In [9]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
hp["label"] = 0
hp["class"] = "high_priority"
hp.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.142707e+10,IssuesEvent,2020-02-03 23:35:24,python/mypy,https://api.github.com/repos/python/mypy,closed,Type inference of Tuples returns object instead,bug false-positive priority-0-high,While interacting with `zip` I encountered a v...,0,high_priority
1,2.655427e+10,IssuesEvent,2023-01-20 10:36:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Unbound type variable false positive with six....,bug topic-type-variables priority-0-high,This code generates a false positive:\r\n```py...,0,high_priority
2,2.656236e+10,IssuesEvent,2023-01-20 16:54:26,python/mypy,https://api.github.com/repos/python/mypy,closed,Unbound type variable false positive with six....,bug topic-type-variables priority-0-high,This code generates a false positive:\r\n```py...,0,high_priority
3,5.439713e+09,IssuesEvent,2017-03-06 14:10:37,python/mypy,https://api.github.com/repos/python/mypy,closed,Crash in super() outside a method,crash priority-0-high,The simplest repro:\r\n```python\r\nclass C:\r...,0,high_priority
4,8.708969e+09,IssuesEvent,2018-12-06 12:35:33,python/mypy,https://api.github.com/repos/python/mypy,closed,TypedDict missing many dict methods,bug false-positive priority-0-high topic-typed...,mypy isn't recognizing methods of TypedDict.\r...,0,high_priority


In [10]:
random["label"] = 1
random["class"] = "not_high_priority" 
random.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.362788e+10,IssuesEvent,2020-09-24 13:13:03,python/mypy,https://api.github.com/repos/python/mypy,opened,Daemon support for --follow-imports=silent,feature,"After #5870 is done, it would be nice to also ...",1,not_high_priority
1,1.363241e+10,IssuesEvent,2020-09-24 19:36:27,python/mypy,https://api.github.com/repos/python/mypy,opened,regression: assignment of 'builtins.type' now ...,bug,\r\n**Bug Report**\r\n\r\n#7963 causes a serio...,1,not_high_priority
2,2.407484e+10,IssuesEvent,2022-09-18 17:03:35,python/mypy,https://api.github.com/repos/python/mypy,opened,stubtest: more concise error for forgotten arg...,feature,**Feature**\r\n\r\nWhen the implementation add...,1,not_high_priority
3,7.238785e+09,IssuesEvent,2018-02-13 15:37:22,python/mypy,https://api.github.com/repos/python/mypy,closed,Typechecking attrs-generated classes,feature needs discussion topic-plugins,[attrs](https://github.com/hynek/attrs) remove...,1,not_high_priority
4,3.912026e+09,IssuesEvent,2016-04-20 08:51:12,python/mypy,https://api.github.com/repos/python/mypy,closed,Improve unification of conditional expressions,bug,I have this snippet of code\r\n\r\n```python\r...,1,not_high_priority


In [11]:
# Drop duplicates by the content of the title
high_priority = hp.drop_duplicates(subset=['title'], keep='last')
high_priority.dropna(inplace=True)
high_priority.reset_index(inplace=True)
high_priority.drop(columns=["index"] , inplace= True)
high_priority["class"].value_counts()

/tmp/ipykernel_166753/1823776403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.dropna(inplace=True)
/tmp/ipykernel_166753/1823776403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.drop(columns=["index"] , inplace= True)


class
high_priority    368
Name: count, dtype: int64

In [12]:
# Drop duplicates by the content of the title
not_high_priority = random.drop_duplicates(subset=['title'], keep='last')
not_high_priority.dropna(inplace=True)
not_high_priority.reset_index(inplace=True)
not_high_priority.drop(columns=["index"] , inplace= True)
not_high_priority["class"].value_counts()

/tmp/ipykernel_166753/3923262596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.dropna(inplace=True)
/tmp/ipykernel_166753/3923262596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.drop(columns=["index"] , inplace= True)


class
not_high_priority    3096
Name: count, dtype: int64

In [13]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          368 non-null    float64
 1   type        368 non-null    object 
 2   created_at  368 non-null    object 
 3   repo        368 non-null    object 
 4   repo_url    368 non-null    object 
 5   action      368 non-null    object 
 6   title       368 non-null    object 
 7   labels      368 non-null    object 
 8   body        368 non-null    object 
 9   label       368 non-null    int64  
 10  class       368 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 31.8+ KB


In [14]:
label_counts = high_priority["class"].value_counts()
label_counts_nhp = not_high_priority["class"].value_counts()
print(label_counts)
not_high_priority_count = label_counts_nhp["not_high_priority"]
print(not_high_priority_count)
hp_count = label_counts["high_priority"]
hp_count

class
high_priority    368
Name: count, dtype: int64
3096


368

In [15]:
not_high_priority = not_high_priority.sample(frac=hp_count/not_high_priority_count, random_state=42)
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
1947,4.343434e+09,IssuesEvent,2016-07-29 01:46:13,python/mypy,https://api.github.com/repos/python/mypy,opened,"Spurious error on Callables returning None, un...",affects/strict-optional,"Under `--strict-optional`, something goes wron...",1,not_high_priority
1309,2.531361e+10,IssuesEvent,2022-11-17 19:33:57,python/mypy,https://api.github.com/repos/python/mypy,opened,Support __call__ on metaclasses,bug,Currently we don't handle `__call__()` on meta...,1,not_high_priority
1606,2.677817e+10,IssuesEvent,2023-01-31 18:49:00,python/mypy,https://api.github.com/repos/python/mypy,closed,Gradual typing with a .lock file,feature,**Feature**\r\n\r\nIn existing untyped project...,1,not_high_priority
2626,2.503724e+10,IssuesEvent,2022-11-04 17:04:02,python/mypy,https://api.github.com/repos/python/mypy,opened,"Using Generic and __new__ in class def, mypy m...",bug,\r\n**Bug Report**\r\n\r\nThis came up in pand...,1,not_high_priority
2440,1.473940e+10,IssuesEvent,2021-01-07 07:07:46,python/mypy,https://api.github.com/repos/python/mypy,closed,Optional with a test still think it could be None,bug,<!--\r\n If you're new to mypy and you're not...,1,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...
1064,1.495171e+10,IssuesEvent,2021-01-26 14:43:02,python/mypy,https://api.github.com/repos/python/mypy,opened,Cannot suppress python_version-related errors ...,bug,**Bug Report**\r\n\r\nIf i have `python_versio...,1,not_high_priority
2856,2.462657e+10,IssuesEvent,2022-10-16 15:47:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Mypy uses installed Python version even when e...,bug,"When running mypy under Python 3.8, I'm gettin...",1,not_high_priority
486,2.429792e+10,IssuesEvent,2022-09-29 11:40:24,python/mypy,https://api.github.com/repos/python/mypy,opened,A function returning TypeVar should receive at...,documentation,**Documentation**\r\n\r\nmypy recently (first ...,1,not_high_priority
2196,2.635768e+10,IssuesEvent,2023-01-11 10:56:55,python/mypy,https://api.github.com/repos/python/mypy,opened,Use mypy to check arrow.get().today() method p...,bug,<!--\r\nIf you're not sure whether what you're...,1,not_high_priority


In [16]:
all_priority = pd.concat([high_priority,not_high_priority] , ignore_index = True)
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
731,1.495171e+10,IssuesEvent,2021-01-26 14:43:02,python/mypy,https://api.github.com/repos/python/mypy,opened,Cannot suppress python_version-related errors ...,bug,**Bug Report**\r\n\r\nIf i have `python_versio...,1,not_high_priority
732,2.462657e+10,IssuesEvent,2022-10-16 15:47:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Mypy uses installed Python version even when e...,bug,"When running mypy under Python 3.8, I'm gettin...",1,not_high_priority
733,2.429792e+10,IssuesEvent,2022-09-29 11:40:24,python/mypy,https://api.github.com/repos/python/mypy,opened,A function returning TypeVar should receive at...,documentation,**Documentation**\r\n\r\nmypy recently (first ...,1,not_high_priority
734,2.635768e+10,IssuesEvent,2023-01-11 10:56:55,python/mypy,https://api.github.com/repos/python/mypy,opened,Use mypy to check arrow.get().today() method p...,bug,<!--\r\nIf you're not sure whether what you're...,1,not_high_priority
735,2.949752e+10,IssuesEvent,2023-06-02 18:20:21,python/mypy,https://api.github.com/repos/python/mypy,opened,Generic function parameter `type[T]` does not ...,bug,### Bug Report\r\n\r\nA generic function with ...,1,not_high_priority


In [17]:
# 
all_priority["label"].value_counts()

label
0    368
1    368
Name: count, dtype: int64

In [18]:
print(all_priority["title"][0])
print(all_priority["body"][0])

Type inference of Tuples returns object instead
While interacting with `zip` I encountered a variant of the following error:
```
error: No overload variant of "zip" matches argument types [builtins.list[builtins.int*], builtins.object*]
```
which is emitted by mypy on the case of
```py
out = list(
    list(zip([1, 2], ordered_item))
    for item in [(1, 2), (4, 5)]
    for ordered_item in [item, tuple(reversed(item))]
    )
print(out)
```
This code runs normally on python 3.6.5, outputting:
```
[[(1, 1), (2, 2)], [(1, 2), (2, 1)], [(1, 4), (2, 5)], [(1, 5), (2, 4)]]
```

## Expected Behavior
Mypy should infer that `ordered_item` is a tuple, or at least an iterable so that `zip` can be applied on it.

It may also be interesting to consider the case where `ordered_item` is heterogenous over different `Iterables` (i.e. not only tuples). Would Iterable be inferred then?

## Actual Behavior
Mypy infers the `object` type on `ordered_item` which you understandably cannot zip.

By explicitly c

In [19]:
# Copy content of body to a new col named text
all_priority["text"] = all_priority["title"] + all_priority["body"]
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class,text
731,1.495171e+10,IssuesEvent,2021-01-26 14:43:02,python/mypy,https://api.github.com/repos/python/mypy,opened,Cannot suppress python_version-related errors ...,bug,**Bug Report**\r\n\r\nIf i have `python_versio...,1,not_high_priority,Cannot suppress python_version-related errors ...
732,2.462657e+10,IssuesEvent,2022-10-16 15:47:13,python/mypy,https://api.github.com/repos/python/mypy,opened,Mypy uses installed Python version even when e...,bug,"When running mypy under Python 3.8, I'm gettin...",1,not_high_priority,Mypy uses installed Python version even when e...
733,2.429792e+10,IssuesEvent,2022-09-29 11:40:24,python/mypy,https://api.github.com/repos/python/mypy,opened,A function returning TypeVar should receive at...,documentation,**Documentation**\r\n\r\nmypy recently (first ...,1,not_high_priority,A function returning TypeVar should receive at...
734,2.635768e+10,IssuesEvent,2023-01-11 10:56:55,python/mypy,https://api.github.com/repos/python/mypy,opened,Use mypy to check arrow.get().today() method p...,bug,<!--\r\nIf you're not sure whether what you're...,1,not_high_priority,Use mypy to check arrow.get().today() method p...
735,2.949752e+10,IssuesEvent,2023-06-02 18:20:21,python/mypy,https://api.github.com/repos/python/mypy,opened,Generic function parameter `type[T]` does not ...,bug,### Bug Report\r\n\r\nA generic function with ...,1,not_high_priority,Generic function parameter `type[T]` does not ...


In [20]:
all_priority["text"][0]

'Type inference of Tuples returns object insteadWhile interacting with `zip` I encountered a variant of the following error:\r\n```\r\nerror: No overload variant of "zip" matches argument types [builtins.list[builtins.int*], builtins.object*]\r\n```\r\nwhich is emitted by mypy on the case of\r\n```py\r\nout = list(\r\n    list(zip([1, 2], ordered_item))\r\n    for item in [(1, 2), (4, 5)]\r\n    for ordered_item in [item, tuple(reversed(item))]\r\n    )\r\nprint(out)\r\n```\r\nThis code runs normally on python 3.6.5, outputting:\r\n```\r\n[[(1, 1), (2, 2)], [(1, 2), (2, 1)], [(1, 4), (2, 5)], [(1, 5), (2, 4)]]\r\n```\r\n\r\n## Expected Behavior\r\nMypy should infer that `ordered_item` is a tuple, or at least an iterable so that `zip` can be applied on it.\r\n\r\nIt may also be interesting to consider the case where `ordered_item` is heterogenous over different `Iterables` (i.e. not only tuples). Would Iterable be inferred then?\r\n\r\n## Actual Behavior\r\nMypy infers the `object` type

In [21]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]
all_priority_subset

,text,label,class
0,Type inference of Tuples returns object instea...,0,high_priority
1,Unbound type variable false positive with six....,0,high_priority
2,Crash in super() outside a methodThe simplest ...,0,high_priority
3,TypedDict missing many dict methodsmypy isn't ...,0,high_priority
4,TypedDict and delMypy disallows `del td[key]` ...,0,high_priority
...,...,...,...
731,Cannot suppress python_version-related errors ...,1,not_high_priority
732,Mypy uses installed Python version even when e...,1,not_high_priority
733,A function returning TypeVar should receive at...,1,not_high_priority
734,Use mypy to check arrow.get().today() method p...,1,not_high_priority


In [22]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_166753/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [23]:
all_priority_subset

,text,label,class,text_str
0,Type inference of Tuples returns object instea...,0,high_priority,Type inference of Tuples returns object instea...
1,Unbound type variable false positive with six....,0,high_priority,Unbound type variable false positive with six....
2,Crash in super() outside a methodThe simplest ...,0,high_priority,Crash in super() outside a methodThe simplest ...
3,TypedDict missing many dict methodsmypy isn't ...,0,high_priority,TypedDict missing many dict methodsmypy isn't ...
4,TypedDict and delMypy disallows `del td[key]` ...,0,high_priority,TypedDict and delMypy disallows `del td[key]` ...
...,...,...,...,...
731,Cannot suppress python_version-related errors ...,1,not_high_priority,Cannot suppress python_version-related errors ...
732,Mypy uses installed Python version even when e...,1,not_high_priority,Mypy uses installed Python version even when e...
733,A function returning TypeVar should receive at...,1,not_high_priority,A function returning TypeVar should receive at...
734,Use mypy to check arrow.get().today() method p...,1,not_high_priority,Use mypy to check arrow.get().today() method p...


In [24]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_166753/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [25]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,type inference of tuples returns object instea...,0
1,unbound type variable false positive with sixw...,0
2,crash in super outside a methodthe simplest re...,0
3,typeddict missing many dict methodsmypy isnt r...,0
4,typeddict and delmypy disallows del tdkey for ...,0
...,...,...
731,cannot suppress pythonversionrelated errors in...,1
732,mypy uses installed python version even when e...,1
733,a function returning typevar should receive at...,1
734,use mypy to check arrowgettoday method prompt ...,1


In [26]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_166753/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_166753/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,type inference of tuples returns object instea...,0
1,unbound type variable false positive with sixw...,0
2,crash in super outside a methodthe simplest re...,0
3,typeddict missing many dict methodsmypy isnt r...,0
4,typeddict and delmypy disallows del tdkey for ...,0
...,...,...
730,cannot suppress pythonversionrelated errors in...,1
731,mypy uses installed python version even when e...,1
732,a function returning typevar should receive at...,1
733,use mypy to check arrowgettoday method prompt ...,1


In [27]:
from sklearn.utils import resample

# Split the dataset into two based on the label
df_majority = priority_label_text[priority_label_text['label'] == 1]
df_minority = priority_label_text[priority_label_text['label'] == 0]

# Undersample the majority class
df_majority_undersampled = resample(df_majority, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority),     # to match minority class
                                     random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset to avoid any ordering bias
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
priority_label_text = df_balanced
priority_label_text["label"].value_counts()

label
1    367
0    367
Name: count, dtype: int64

In [28]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_mypy_testset.csv"
priority_label_text.to_csv(file_name, index=False)

In [29]:
pri = pd.read_csv(file_name)
pri

,text_clean,label
0,callable with a concrete subtype as a paramete...,1
1,regression lambda within class body causes a c...,0
2,ignore specific importsfeature apologies if th...,1
3,performance regression 0770 0780after upgradin...,0
4,support type aliases in finegrained incrementa...,0
...,...,...
729,slow incremental run when single module has er...,0
730,metaclass doesnt work across import cyclei hav...,0
731,new semantic analyser establish and document t...,0
732,mypy doesnt realize none is an instance of typ...,1
